This notebook is created as a reference for SeqMetrics tests.
Since the SeqMetrics contain functions which are present in various other libraries, putting them as dependency while running automatic tests will
make testing quite difficult. Since it will make tests depends upon
a lot of libraries, some of which are no more maintained. Therefore,
in this notebook, we have installed those libraries and called those functions.
Some of the tests in SeqMetrics are compared against outputs of the functions
called in this notebook. This notebook is mainly intended for developers and reviewers of SeqMetrics library.


In [1]:
!pip install hydroeval

In [2]:
!pip install NeuralHydrology

In [3]:
!pip install HydroErr

In [4]:
!pip install SkillMetrics

In [5]:
!pip install RegscorePy

In [6]:
!pip install sktime

In [7]:
import datetime

import numpy as np

import xarray as xr

import pandas as pd

import hydroeval
from hydroeval import evaluator

import neuralhydrology

import HydroErr

import skill_metrics

import RegscorePy

import sktime

print(datetime.datetime.now().strftime("%d %B %Y %H:%M:%S"))
print('xarray version: ', xr.__version__)
print('np version: ', np.__version__)
print('pd version: ', pd.__version__)
print('hydroeval version: ', hydroeval.__version__)
print('sktime: ', sktime.__version__)

12 May 2024 08:24:28
xarray version:  2023.7.0
np version:  1.25.2
pd version:  2.0.3
hydroeval version:  0.1.0
sktime:  0.29.0


In [8]:
random_state = np.random.RandomState(seed=313)

t11 = random_state.random(100)
p11 = random_state.random(100)

t11.sum(), p11.sum()

(50.99521786026271, 57.734075639710824)

In [9]:
t_large = random_state.random(100) * 1e7
p_large = random_state.random(100) * 1e7
t_large.sum(), p_large.sum()

(464398965.6802617, 476923506.11901504)

In [10]:
t_nan = t11.copy()
p_nan = p11.copy()
nan_indices = random_state.choice(range(100), 10, replace=False)
t_nan[nan_indices] = np.nan
nan_indices = random_state.choice(range(100), 10, replace=False)
p_nan[nan_indices] = np.nan

np.nansum(t_nan), np.nansum(p_nan)

(45.8916112408714, 50.981912295587875)

In [11]:
t_neg = random_state.randint(-100, 100, 100)
p_neg = random_state.randint(-100, 100, 100)

t_neg.sum(), p_neg.sum()

(-538, 545)

In [12]:
"""
The helper functions in this cell are copied from https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9
"""
EPSILON = 1e-10


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted


def _percentage_error(actual: np.ndarray, predicted: np.ndarray):
    """
    Percentage error
    Note: result is NOT multiplied by 100
    """
    return _error(actual, predicted) / (actual + EPSILON)


def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    """ Naive forecasting method which just repeats previous samples """
    return actual[:-seasonality]


def _relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark
        return _error(actual[seasonality:], predicted[seasonality:]) /\
               (_error(actual[seasonality:], _naive_forecasting(actual, seasonality)) + EPSILON)

    return _error(actual, predicted) / (_error(actual, benchmark) + EPSILON)

def _bounded_relative_error(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Bounded Relative Error """
    if benchmark is None or isinstance(benchmark, int):
        # If no benchmark prediction provided - use naive forecasting
        if not isinstance(benchmark, int):
            seasonality = 1
        else:
            seasonality = benchmark

        abs_err = np.abs(_error(actual[seasonality:], predicted[seasonality:]))
        abs_err_bench = np.abs(_error(actual[seasonality:], _naive_forecasting(actual, seasonality)))
    else:
        abs_err = np.abs(_error(actual, predicted))
        abs_err_bench = np.abs(_error(actual, benchmark))

    return abs_err / (abs_err + abs_err_bench + EPSILON)


def _geometric_mean(a, axis=0, dtype=None):
    """ Geometric mean """
    if not isinstance(a, np.ndarray):  # if not an ndarray object attempt to convert it
        log_a = np.log(np.array(a, dtype=dtype))
    elif dtype:  # Must change the default dtype allowing array type
        if isinstance(a, np.ma.MaskedArray):
            log_a = np.log(np.ma.asarray(a, dtype=dtype))
        else:
            log_a = np.log(np.asarray(a, dtype=dtype))
    else:
        log_a = np.log(a)
    return np.exp(log_a.mean(axis=axis))


def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))

def mape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Absolute Percentage Error
    Properties:
        + Easy to interpret
        + Scale independent
        - Biased, not symmetric
        - Undefined when actual[t] == 0
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.abs(_percentage_error(actual, predicted)))



### mda

In [13]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L259

def mda(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - actual[:-1])).astype(int))


mda(t11, p11), mda(t_large, p_large), mda(t_nan, p_nan), mda(t_neg, p_neg)

(0.696969696969697, 0.7171717171717171, 0.494949494949495, 0.7171717171717171)

### mbrae

In [14]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L248

def mbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Bounded Relative Absolute Error """
    return np.mean(_bounded_relative_error(actual, predicted, benchmark))

mbrae(t11, p11), mbrae(t_large, p_large), mbrae(t_nan, p_nan), mbrae(t_neg, p_neg)

(0.46659593775205116, 0.5036144959534816, nan, 0.4330684929959228)

### umbrae

In [15]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L253
def umbrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Unscaled Mean Bounded Relative Absolute Error """
    __mbrae = mbrae(actual, predicted, benchmark)
    return __mbrae / (1 - __mbrae)

umbrae(t11, p11), umbrae(t_large, p_large), umbrae(t_nan, p_nan), umbrae(t_neg, p_neg)

(0.8747513766311694, 1.014563261513547, nan, 0.7638815053417172)

### gmrae

In [16]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L243

def gmrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Geometric Mean Relative Absolute Error """
    return _geometric_mean(np.abs(_relative_error(actual, predicted, benchmark)))

gmrae(t11, p11), gmrae(t_large, p_large), gmrae(t_nan, p_nan), gmrae(t_neg, p_neg)

<ipython-input-12-26b8d5991418>:66: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)


(0.79938390310645, 1.1057529056395325, nan, 0.0)

### mdrae

In [17]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L238

def mdrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Median Relative Absolute Error """
    return np.median(np.abs(_relative_error(actual, predicted, benchmark)))

mdrae(t11, p11), mdrae(t_large, p_large), mdrae(t_nan, p_nan), mdrae(t_neg, p_neg)

(0.9086455067666214, 0.941921124439974, nan, 0.7777777777768176)

### mrae

In [18]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L233

def mrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Absolute Error """
    return np.mean(np.abs(_relative_error(actual, predicted, benchmark)))

mrae(t11, p11), mrae(t_large, p_large), mrae(t_nan, p_nan), mrae(t_neg, p_neg)

(2.5711621568850163, 17.10021183277437, nan, 1.6932857266200108)

### rae

In [19]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L228

def rae(actual: np.ndarray, predicted: np.ndarray):
    """ Relative Absolute Error (aka Approximation Error) """
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)

rae(t11, p11), rae(t_large, p_large), rae(t_nan, p_nan), rae(t_neg, p_neg)

(1.3287945409387383, 1.340411324601443, nan, 1.2510189384046542)

### mre

In [20]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L223

def mre(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Error """
    return np.mean(_relative_error(actual, predicted, benchmark))

mre(t11, p11), mre(t_large, p_large), mre(t_nan, p_nan), mre(t_neg, p_neg)

(0.5101139564068491, -1.5538232747884768, nan, 0.05273425381880722)

In [21]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L223

def rse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Relative Squared Error """
    return np.sum(np.square(actual - predicted)) / np.sum(np.square(actual - np.mean(actual)))

rse(t11, p11), rse(t_large, p_large), rse(t_nan, p_nan), rse(t_neg, p_neg)

(2.0683722517498735, 1.9614489156992236, nan, 1.828654286138622)

### rrse

In [22]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L223

def rrse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Relative Squared Error """
    return np.sqrt(np.sum(np.square(actual - predicted)) / np.sum(np.square(actual - np.mean(actual))))

rrse(t11, p11), rrse(t_large, p_large), rrse(t_nan, p_nan), rrse(t_neg, p_neg)

(1.4381836641228662, 1.4005173742939512, nan, 1.3522774442172072)

### inrse

In [23]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L213

def inrse(actual: np.ndarray, predicted: np.ndarray):
    """ Integral Normalized Root Squared Error """
    return np.sqrt(np.sum(np.square(_error(actual, predicted))) / np.sum(np.square(actual - np.mean(actual))))

inrse(t11, p11), inrse(t_large, p_large), inrse(t_nan, p_nan), inrse(t_neg, p_neg)

(1.4381836641228662, 1.4005173742939512, nan, 1.3522774442172072)

### rmsse

In [24]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L207

def rmsse(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """ Root Mean Squared Scaled Error """
    q = np.abs(_error(actual, predicted)) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))
    return np.sqrt(np.mean(np.square(q)))

rmsse(t11,p11), rmsse(t_large, p_large), rmsse(t_nan, p_nan), rmsse(t_neg, p_neg)

(1.2234619716320643, 1.217492218321228, nan, 1.0319875236848162)

### me

In [25]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L86

def me(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Error """
    return np.mean(_error(actual, predicted))

me(t11, p11), me(t_large, p_large), me(t_nan, p_nan), me(t_neg, p_neg)

(-0.06738857779448111, -125245.40438753393, nan, -10.83)

### rmse

In [26]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L76

def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))

rmse(t11, p11), rmse(t_large, p_large), rmse(t_nan, p_nan), rmse(t_neg, p_neg)

(0.40289487147518754, 4183830.0609148364, nan, 79.05649878409744)

### nrmse

In [27]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L81

def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (actual.max() - actual.min())

nrmse(t11, p11), nrmse(t_large, p_large), nrmse(t_nan, p_nan), nrmse(t_neg, p_neg)

(0.4081874143525102, 0.4236625368431997, nan, 0.4033494835923339)

### mpe

In [28]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L112

def mpe(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Percentage Error """
    return np.mean(_percentage_error(actual, predicted))

mpe(t11, p11), mpe(t_large, p_large), mpe(t_nan, p_nan), mpe(t_neg, p_neg)

(-42.20843064537674, -2.2416867347068568, nan, -0.5080812552030656)

### gmae

In [29]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L102

def gmae(actual: np.ndarray, predicted: np.ndarray):
    """ Geometric Mean Absolute Error """
    return _geometric_mean(np.abs(_error(actual, predicted)))

gmae(t11, p11), gmae(t_large, p_large), gmae(t_nan, p_nan), gmae(t_neg, p_neg)

<ipython-input-12-26b8d5991418>:66: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)


(0.19423992928498718, 2449891.1923250267, nan, 0.0)

### mdape

In [30]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L132

def mdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(np.abs(_percentage_error(actual, predicted)))

mdape(t11, p11), mdape(t_large, p_large), mdape(t_nan, p_nan), mdape(t_neg, p_neg)

(0.513246349701827, 0.7567708188562414, nan, 1.4614370468057651)

### smape

In [31]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L141

def smape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Mean Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))

smape(t11, p11), smape(t_large, p_large), smape(t_nan, p_nan), smape(t_neg, p_neg)

(0.7028826489278417, 0.8728058345172697, nan, 1.2665321112176788)

### smdape

In [32]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L150

def smdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))

smdape(t11, p11), smdape(t_large, p_large), smdape(t_nan, p_nan), smdape(t_neg, p_neg)

(0.5999121382638821, 0.7801452772689506, nan, 1.4883449883428177)

### maape

In [33]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L159

def maape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Arctangent Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))

maape(t11, p11), maape(t_large, p_large), maape(t_nan, p_nan), maape(t_neg, p_neg)

(0.5828454707567975, 0.6788115353385473, nan, 0.8399216450191015)

### mase

In [34]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L168

def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))

mase(t11, p11), mase(t_large, p_large), mase(t_nan, p_nan), mase(t_neg, p_neg)

(0.9609397361653512, 1.0143742396253592, nan, 0.8253916139240506)

### std_ae
renamed as norm_ae in seqmetrics


In [35]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L177

def std_ae(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Error """
    __mae = mae(actual, predicted)
    return np.sqrt(np.sum(np.square(_error(actual, predicted) - __mae))/(len(actual) - 1))

std_ae(t11, p11), std_ae(t_large, p_large), std_ae(t_nan, p_nan), std_ae(t_neg, p_neg)

(0.5551510970200795, 5553104.916104561, nan, 108.32762082840846)

### std_ape

renamed as norm_ape in seqmetrics

In [36]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L183

def std_ape(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Percentage Error """
    __mape = mape(actual, predicted)
    return np.sqrt(np.sum(np.square(_percentage_error(actual, predicted) - __mape))/(len(actual) - 1))

std_ape(t11, p11), std_ape(t_large, p_large), std_ape(t_nan, p_nan), std_ape(t_neg, p_neg)

(404.0632278812062, 10.468496670954119, nan, 11.097150018412195)

### rmspe

In [37]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L189

def rmspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Mean Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.mean(np.square(_percentage_error(actual, predicted))))

rmspe(t11, p11), rmspe(t_large, p_large), rmspe(t_nan, p_nan), rmspe(t_neg, p_neg)

(395.25283254969173, 9.399644256990216, nan, 10.206148170735952)

### rmdspe

In [38]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L198

def rmdspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Median Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.median(np.square(_percentage_error(actual, predicted))))

rmdspe(t11, p11), rmdspe(t_large, p_large), rmdspe(t_nan, p_nan), rmdspe(t_neg, p_neg)

(0.5133222853161394, 0.7567716892334615, nan, 1.4614383848209451)

### NSE

In [39]:
from hydroeval import nse

# Calculate NSE
evaluator(nse, p11, t11)[0], nse(p_large, t_large), nse(p_nan, t_nan), nse(p_neg, t_neg)

(-1.0683722517498735, -0.9614489156992236, nan, -0.8286542861386219)

### KGE

In [40]:
from hydroeval import kge

# Calculate KGE
evaluator(kge, p11, t11)[0].item(), kge(p_large, t_large)[0].item(), kge(p_nan, t_nan)[0], kge(p_neg, t_neg)[0].item()

(0.008970625237195717, 0.038626769079834755, array([nan]), -1.193416946870867)

### alpha_nse

In [41]:

from neuralhydrology.evaluation.metrics import alpha_nse

alpha_nse(xr.DataArray(t11), xr.DataArray(p11)), alpha_nse(xr.DataArray(t_large), xr.DataArray(p_large)), alpha_nse(xr.DataArray(t_nan), xr.DataArray(p_nan)), alpha_nse(xr.DataArray(t_neg), xr.DataArray(p_neg))

(1.0235046034233621, 1.0196280911618452, 0.9959736532709084, 1.02998712689571)

### beta_nse

In [42]:
from neuralhydrology.evaluation.metrics import beta_nse

beta_nse(xr.DataArray(t11), xr.DataArray(p11)), beta_nse(xr.DataArray(t_large), xr.DataArray(p_large)), beta_nse(xr.DataArray(t_nan), xr.DataArray(p_nan)), beta_nse(xr.DataArray(t_neg), xr.DataArray(p_neg))

(0.2405519617999516,
 0.041925308232251185,
 0.28770418762004457,
 0.18524934630444692)

### nse_mod

In [43]:

from HydroErr import nse_mod
nse_mod(p11, t11), nse_mod(p_large, t_large), nse_mod(p_nan, t_nan), nse_mod(p_neg, t_neg)

/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:6248: UserWarning: Row(s) [ 5  8  9 11 12 26 27 36 42 59 62 63 65 67 74 79 84 89 98] contained NaN values and the row(s) have been removed (Rows are zero indexed).
  warnings.warn("Row(s) {} contained NaN values and the row(s) have been "


(-0.32879454094431804,
 -0.3404113246014431,
 -0.30533623352042705,
 -0.2510189384046788)

### relative nse

In [44]:
from HydroErr import nse_rel

nse_rel(p11, t11), nse_rel(p_large, t_large), nse_rel(p_nan, t_nan), nse_rel(p_neg, t_neg)

(-517670.8159599439,
 -212.51788168937344,
 -957.2667493440044,
 0.11784531614883476)

### nse_bound

In [45]:
from hydroeval import nse_c2m

nse_c2m(p11, t11), nse_c2m(p_large, t_large), nse_c2m(p_nan, t_nan), nse_c2m(p_neg, t_neg)

(-0.34818860428052284, -0.324654904767188, nan, -0.29295000460088483)

### kge_bound

In [46]:

from hydroeval import kge_c2m

kge_c2m(p11, t11), kge_c2m(p_large, t_large), kge_c2m(p_nan, t_nan), kge_c2m(p_neg, t_neg)

(array([0.00450552]), array([0.01969374]), array([nan]), array([-0.3737116]))

### kge_mod

In [47]:

from HydroErr import kge_2012

kge_2012(p11, t11), kge_2012(p_large, t_large), kge_2012(p_nan, t_nan), kge_2012(p_neg, t_neg)

(0.004612979178136856,
 0.03880057852421015,
 -0.01881219864144601,
 -1.9795119132448913)

### kge_np

In [48]:

from hydroeval import kgenp

evaluator(kgenp, p11, t11)[0], kgenp(p_large, t_large)[0], kgenp(p_nan, t_nan)[0], kgenp(p_neg, t_neg)[0]

(array([-0.00671877]), array([0.04099218]), array([nan]), array([-1.25351754]))

### pbias

In [49]:

from hydroeval import pbias

evaluator(pbias, p11, t11), pbias(p_large, t_large), pbias(p_nan, t_nan), pbias(p_neg, t_neg)

(array([-13.21468573]), -2.696935472370653, nan, 201.3011152416357)

In [50]:
pbias(p11, t11)

-13.21468573369753

In [51]:
from skill_metrics import bias_percent

bias_percent(p11, t11), bias_percent(p_large, t_large), bias_percent(p_nan, t_nan), bias_percent(p_neg, t_neg)

(13.214685733697538, 2.6969354723706473, nan, 201.3011152416357)

### bias

In [52]:
from skill_metrics import bias

bias(p11, t11), bias(p_large, t_large), bias(p_nan, t_nan), bias(p_neg, t_neg)

(0.06738857779448115, 125245.40438753366, nan, 10.83)

### gmae

In [53]:

# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L102

def gmae(actual: np.ndarray, predicted: np.ndarray):
    """ Geometric Mean Absolute Error """
    return _geometric_mean(np.abs(_error(actual, predicted)))

gmae(t11, p11), gmae(t_large, p_large), gmae(t_nan, p_nan), gmae(t_neg, p_neg)

<ipython-input-12-26b8d5991418>:66: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)


(0.19423992928498718, 2449891.1923250267, nan, 0.0)

### mase

In [54]:

# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L168

def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))

mase(t11, p11), mase(t_large, p_large), mase(t_nan, p_nan), mase(t_neg, p_neg)

(0.9609397361653512, 1.0143742396253592, nan, 0.8253916139240506)

### fdc_fhv

In [55]:
from neuralhydrology.evaluation.metrics import fdc_fhv

fdc_fhv(xr.DataArray(t11), xr.DataArray(p11)), fdc_fhv(xr.DataArray(t_large), xr.DataArray(p_large)), fdc_fhv(xr.DataArray(t_nan), xr.DataArray(p_nan)), fdc_fhv(xr.DataArray(t_neg), xr.DataArray(p_neg))

(1.5933766138626262, -2.2166403701887254, 1.5933766138626262, 0.0)

### fdc_flv

In [56]:
from neuralhydrology.evaluation.metrics import fdc_flv

fdc_flv(xr.DataArray(t11), xr.DataArray(p11)), fdc_flv(xr.DataArray(t_large), xr.DataArray(p_large)), fdc_flv(xr.DataArray(t_nan), xr.DataArray(p_nan)), fdc_flv(xr.DataArray(t_neg), xr.DataArray(p_neg))

/usr/local/lib/python3.10/dist-packages/neuralhydrology/evaluation/metrics.py:526: RuntimeWarning: invalid value encountered in log
  obs = np.log(obs)
/usr/local/lib/python3.10/dist-packages/neuralhydrology/evaluation/metrics.py:527: RuntimeWarning: divide by zero encountered in log
  sim = np.log(sim)
/usr/local/lib/python3.10/dist-packages/neuralhydrology/evaluation/metrics.py:530: RuntimeWarning: invalid value encountered in subtract
  qsl = np.sum(sim - sim.min())


(32.64817835760714, -113.29167126784779, -35.650939085951364, nan)

### maape

In [57]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L159

def maape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Arctangent Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))

maape(t11, p11), maape(t_large, p_large), maape(t_nan, p_nan), maape(t_neg, p_neg)

(0.5828454707567975, 0.6788115353385473, nan, 0.8399216450191015)

### lm_index

In [58]:
from HydroErr import lm_index

lm_index(p11, t11), lm_index(p_large, t_large), lm_index(p_nan, t_nan), lm_index(p_neg, t_neg)

/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:6248: UserWarning: Row(s) [ 5  8  9 11 12 26 27 36 42 59 62 63 65 67 74 79 84 89 98] contained NaN values and the row(s) have been removed (Rows are zero indexed).
  warnings.warn("Row(s) {} contained NaN values and the row(s) have been "


(-0.32879454094431804,
 -0.3404113246014431,
 -0.30533623352042705,
 -0.2510189384046788)

### kgenp_bound

In [59]:
from hydroeval import kgenp_c2m

In [60]:
evaluator(kgenp_c2m, p11, t11), kgenp_c2m(p_large, t_large), kgenp_c2m(p_nan, t_nan), kgenp_c2m(p_neg, t_neg)

(array([-0.00334814]), array([0.02092497]), array([nan]), array([-0.38528071]))

### kgeprime_c2m

In [61]:
from hydroeval import kgeprime_c2m

In [62]:
evaluator(kgeprime_c2m, p11, t11), kgeprime_c2m(p_large, t_large), kgeprime_c2m(p_nan, t_nan), kgeprime_c2m(p_neg, t_neg)

(array([0.00231182]), array([0.01978411]), array([nan]), array([-0.4974258]))

### mle

In [63]:
from HydroErr import mle

In [64]:
mle(p11, t11), mle(p_large, t_large), mle(p_nan, t_nan), mle(p_neg, t_neg)

/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:346: RuntimeWarning: invalid value encountered in log1p
  sim_log = np.log1p(simulated_array)
/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:347: RuntimeWarning: divide by zero encountered in log1p
  obs_log = np.log1p(observed_array)
/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:347: RuntimeWarning: invalid value encountered in log1p
  obs_log = np.log1p(observed_array)


(0.0438958324374804, -0.03628518397994352, 0.0549303833066667, nan)

### modified agreement of index

In [65]:
from HydroErr import dmod

In [66]:
dmod(p11, t11), dmod(p_large, t_large), dmod(p_nan, t_nan), dmod(p_neg, t_neg)

(0.36018092524466827,
 0.3388950675310263,
 0.35955299749868597,
 0.40008576980295707)

### mpe

In [67]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L112

def mpe(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Percentage Error """
    return np.mean(_percentage_error(actual, predicted))

In [68]:
mpe(t11, p11), mpe(t_large, p_large), mpe(t_nan, p_nan), mpe(t_neg, p_neg)

(-42.20843064537674, -2.2416867347068568, nan, -0.5080812552030656)

### mrae

In [69]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L233

def mrae(actual: np.ndarray, predicted: np.ndarray, benchmark: np.ndarray = None):
    """ Mean Relative Absolute Error """
    return np.mean(np.abs(_relative_error(actual, predicted, benchmark)))

mrae(t11, p11), mrae(t_large, p_large), mrae(t_nan, p_nan), mrae(t_neg, p_neg)

(2.5711621568850163, 17.10021183277437, nan, 1.6932857266200108)

### nrmse

In [70]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L81

def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (actual.max() - actual.min())

nrmse(t11, p11), nrmse(t_large, p_large), nrmse(t_nan, p_nan), nrmse(t_neg, p_neg)

(0.4081874143525102, 0.4236625368431997, nan, 0.4033494835923339)

### nrmse_ipercentile

In [71]:
from HydroErr import nrmse_iqr

nrmse_iqr(p11, t11), nrmse_iqr(p_large, t_large), nrmse_iqr(p_nan, t_nan), nrmse_iqr(p_neg, t_neg)

(0.8187123709758822,
 0.7724872344273039,
 0.7918933678745431,
 0.7675388231465771)

### nrmse_mean

In [72]:
from HydroErr import nrmse_mean

nrmse_mean(p11, t11), nrmse_mean(p_large, t_large), nrmse_mean(p_nan, t_nan), nrmse_mean(p_neg, t_neg)

(0.790064026354788,
 0.9009128723588501,
 0.8193408657286905,
 -14.694516502620342)

### acc

In [73]:
from HydroErr import acc

acc(p11, t11), acc(p_large, t_large), acc(p_nan, t_nan), acc(p_neg, t_neg)

(0.0179208383645756,
 0.038813542968754375,
 0.0048403609085283655,
 0.12809439248837096)

### ve

In [74]:
from HydroErr import ve

ve(p11, t11), ve(p_large, t_large), ve(p_nan, t_nan), ve(p_neg, t_neg)

(0.3794625949621073, 0.2493891984559775, 0.3563569652466473, 12.75278810408922)

### watt_m

In [75]:
from HydroErr import watt_m

watt_m(p11, t11), watt_m(p_large, t_large), watt_m(p_nan, t_nan), watt_m(p_neg, t_neg)

(0.017290316806567577,
 0.03105683026117264,
 0.010510101645599523,
 0.08672078755135931)

### skill score murphy

In [76]:
from skill_metrics import skill_score_murphy

skill_score_murphy(p11, t11), skill_score_murphy(p_large, t_large), skill_score_murphy(p_nan, t_nan), skill_score_murphy(p_neg, t_neg)

(-1.0476885292323743, -0.9418344265422312, nan, -0.8103677432772358)

### sid

In [77]:
from HydroErr import sid

sid(p11, t11), sid(p_large, t_large), sid(p_nan, t_nan), sid(p_neg, t_neg)

/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:3768: RuntimeWarning: invalid value encountered in log10
  second1 = np.log10(observed_array) - np.log10(np.mean(observed_array))
/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:3769: RuntimeWarning: invalid value encountered in log10
  second2 = np.log10(simulated_array) - np.log10(np.mean(simulated_array))


(43.71192101756139, 51.90130475363692, 34.09217843319721, nan)

### smdape

In [78]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L150

def smdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Symmetric Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(2.0 * np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) + EPSILON))

smdape(t11, p11), smdape(t_large, p_large), smdape(t_nan, p_nan), smdape(t_neg, p_neg)

(0.5999121382638821, 0.7801452772689506, nan, 1.4883449883428177)

### smape

In [79]:
from HydroErr import smape2

smape2(p11, t11), smape2(p_large, t_large), smape2(p_nan, t_nan), smape2(p_neg, t_neg)

(70.28826490215243, 87.28058345172697, 71.12256546064978, 425.5674491975244)

### sc

In [80]:
from HydroErr import sc

sc(p11, t11), sc(p_large, t_large), sc(p_nan, t_nan), sc(p_neg, t_neg)

(1.5526934811208075, 1.5315806771993858, 1.5658954417562418, 1.441044282467568)

### sa

In [81]:
from HydroErr import sa

sa(p11, t11), sa(p_large, t_large), sa(p_nan, t_nan), sa(p_neg, t_neg)

(0.6618474080345743, 0.7666906960815938, 0.6732143252992395, 1.450447074951836)

### rmspe

In [82]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L189

def rmspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Mean Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.mean(np.square(_percentage_error(actual, predicted))))

rmspe(t11, p11), rmspe(t_large, p_large), rmspe(t_nan, p_nan), rmspe(t_neg, p_neg)

(395.25283254969173, 9.399644256990216, nan, 10.206148170735952)

### relative agreement index

In [83]:
from HydroErr import drel

drel(p11, t11), drel(p_large, t_large), drel(p_nan, t_nan), drel(p_neg, t_neg)

(-139396.49261170527,
 -59.968928280632056,
 -257.2099156747244,
 0.7644847163377675)

### refined agreement index

In [84]:
from HydroErr import dr

dr(p11, t11), dr(p_large, t_large), dr(p_nan, t_nan), dr(p_neg, t_neg)

(0.335602729527841,
 0.32979433769927846,
 0.3473318832397865,
 0.3744905307976606)

### rmdspe

In [85]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L198

def rmdspe(actual: np.ndarray, predicted: np.ndarray):
    """
    Root Median Squared Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.sqrt(np.median(np.square(_percentage_error(actual, predicted))))

rmdspe(t11, p11), rmdspe(t_large, p_large), rmdspe(t_nan, p_nan), rmdspe(t_neg, p_neg)

(0.5133222853161394, 0.7567716892334615, nan, 1.4614383848209451)

### normalized ape

In [86]:
def mape(actual: np.ndarray, predicted: np.ndarray):
    """
    Mean Absolute Percentage Error
    Properties:
        + Easy to interpret
        + Scale independent
        - Biased, not symmetric
        - Undefined when actual[t] == 0
    Note: result is NOT multiplied by 100
    """
    return np.mean(np.abs(_percentage_error(actual, predicted)))

# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L183
def std_ape(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Percentage Error """
    __mape = mape(actual, predicted)
    return np.sqrt(np.sum(np.square(_percentage_error(actual, predicted) - __mape))/(len(actual) - 1))

std_ape(t11, p11), std_ape(t_large, p_large), std_ape(t_nan, p_nan), std_ape(t_neg, p_neg)

(404.0632278812062, 10.468496670954119, nan, 11.097150018412195)

### normalized ae

In [87]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L177

def std_ae(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Absolute Error """
    __mae = mae(actual, predicted)
    return np.sqrt(np.sum(np.square(_error(actual, predicted) - __mae))/(len(actual) - 1))

std_ae(t11, p11), std_ae(t_large, p_large), std_ae(t_nan, p_nan), std_ae(t_neg, p_neg)

(0.5551510970200795, 5553104.916104561, nan, 108.32762082840846)

### normalized euclid distance

In [88]:
from HydroErr import ned

ned(p11, t11), ned(p_large, t_large), ned(p_nan, t_nan), ned(p_neg, t_neg)

(7.338597737626875, 8.885308084386528, 6.7372058740579615, 164.68781514254727)

### euclid distance

In [89]:
from HydroErr import ed

ed(p11, t11), ed(p_large, t_large), ed(p_nan, t_nan), ed(p_neg, t_neg)

(4.028948714751875, 41838300.60914836, 3.727612385864465, 790.5649878409744)

# relative rmse

In [90]:
# https://stackoverflow.com/a/41749834
# https://stackoverflow.com/q/69360839
# https://gist.github.com/AayushSameerShah/baeedbbb52fe808874d6debb7e786183


### mean variance

In [91]:
from HydroErr import mean_var

mean_var(p11, t11), mean_var(p_large, t_large), mean_var(p_nan, t_nan), mean_var(p_neg, t_neg)

/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:6142: RuntimeWarning: divide by zero encountered in log1p
  return np.var(np.log1p(observed_array) - np.log1p(simulated_array))
/usr/local/lib/python3.10/dist-packages/HydroErr/HydroErr.py:6142: RuntimeWarning: invalid value encountered in log1p
  return np.var(np.log1p(observed_array) - np.log1p(simulated_array))


(0.07449144510570738, 2.5613322338568736, 0.0773070071169125, nan)

### median error

In [92]:
from HydroErr import mde

mde(p11, t11), mde(p_large, t_large), mde(p_nan, t_nan), mde(p_neg, t_neg)

(0.0313854202641316, 325158.3003595751, 0.05103081484042016, 13.5)

### mdape

In [93]:
# https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9#file-forecasting_metrics-py-L132

def mdape(actual: np.ndarray, predicted: np.ndarray):
    """
    Median Absolute Percentage Error
    Note: result is NOT multiplied by 100
    """
    return np.median(np.abs(_percentage_error(actual, predicted)))

mdape(t11, p11), mdape(t_large, p_large), mdape(t_nan, p_nan), mdape(t_neg, p_neg)

(0.513246349701827, 0.7567708188562414, nan, 1.4614370468057651)

### Mielke Berry R

In [94]:
from HydroErr import mb_r

mb_r(p11, t11), mb_r(p_large, t_large), mb_r(p_nan, t_nan), mb_r(p_neg, t_neg)

(0.04444743269492335,
 -0.0012306836021478418,
 0.03938975589826077,
 0.08496259099000014)

### aic

In [95]:
from RegscorePy import aic

In [96]:
aic.aic(t11, p11, p=1), aic.aic(t_large, p_large, p=1)#, aic.aic(t_nan, p_nan, p=1), aic.aic(t_neg, p_neg, p=1)

(-179.8159231784877, 3051.347533497176)

### bic

In [97]:
import collections

collections.Sequence = list   # because bic function uses collections.Sequence

from RegscorePy import bic

bic.bic(y=t11, y_pred=p11, p=1), bic.bic(y=t_large, y_pred=p_large, p=1), bic.bic(y=t_nan, y_pred=p_nan, p=1)#, bic.bic(y=t_neg, y_pred=p_neg, p=1)

(-177.2107529924996, 3053.952703683164, nan)

### wmape

In [98]:
# https://stackoverflow.com/a/54833202/5982232

def wmape(actual, forecast):
    # Take a series (actual) and a dataframe (forecast) and calculate wmape
    # for each forecast. Output shape is (1, num_forecasts)

    # Convert to numpy arrays for broadasting
    forecast = np.array(forecast.values)
    actual=np.array(actual.values).reshape((-1, 1))

    # Make an array of mape (same shape as forecast)
    se_mape = abs(actual-forecast)/actual

    # Calculate sum of actual values
    ft_actual_sum = actual.sum(axis=0)

    # Multiply the actual values by the mape
    se_actual_prod_mape = actual * se_mape

    # Take the sum of the product of actual values and mape
    # Make sure to sum down the rows (1 for each column)
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum(axis=0)

    # Calculate the wmape for each forecast and return as a dictionary
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum
    return {f'Forecast_{i+1}_wmape': wmape for i, wmape in enumerate(ft_wmape_forecast)}


wmape(pd.Series(t11), pd.DataFrame(p11)), wmape(pd.DataFrame(t_large), pd.DataFrame(p_large)), wmape(pd.DataFrame(t_nan), pd.DataFrame(p_nan)), wmape(pd.DataFrame(t_neg), pd.DataFrame(p_neg))

({'Forecast_1_wmape': 0.6205374050378927},
 {'Forecast_1_wmape': 0.7506108015440225},
 {'Forecast_1_wmape': nan},
 {'Forecast_1_wmape': -11.75278810408922})

### irmse

In [99]:
from HydroErr import irmse

irmse(p11, t11), irmse(p_large, t_large), irmse(p_nan, t_nan), irmse(p_neg, t_neg)

(0.9954807723243245,
 0.9923269981666207,
 1.0529524887988122,
 0.8861674491048979)

### CronbachAlpha

In [100]:
# https://stackoverflow.com/a/20799687

def CronbachAlpha(itemscores):
    itemscores = np.asarray(itemscores)
    itemvars = itemscores.var(axis=1, ddof=1)
    tscores = itemscores.sum(axis=0)
    nitems = len(itemscores)

    return nitems / (nitems-1.) * (1 - itemvars.sum() / tscores.var(ddof=1))


CronbachAlpha(np.stack([t11, p11])), CronbachAlpha(np.stack([t_large, p_large])), CronbachAlpha(np.stack([t_nan, p_nan])), CronbachAlpha(np.stack([t_neg, p_neg]))

(0.03555058748735895, 0.07543930266724952, nan, 0.22904130861912253)

### aitchison distance

In [101]:
# https://github.com/hallamlab/utilities/blob/master/SparCC/lib/distances.py#L47

def aitchison(x,y, center = 'mean'):
    lx = np.log(x)
    ly = np.log(y)
    if center == 'mean':     m = np.mean
    elif center == 'median': m = np.median
    clr_x = lx - m(lx)
    clr_y = ly - m(ly)
    d = ( sum((clr_x-clr_y)**2) )**0.5
    return d


aitchison(t11, p11), aitchison(t_large, p_large), aitchison(t_nan, p_nan), aitchison(t_neg, p_neg)

<ipython-input-101-97a490b22f00>:4: RuntimeWarning: invalid value encountered in log
  lx = np.log(x)
<ipython-input-101-97a490b22f00>:5: RuntimeWarning: invalid value encountered in log
  ly = np.log(y)


(16.326288844358846, 16.004244764764962, nan, nan)

### JS

In [102]:
# https://github.com/hallamlab/utilities/blob/master/SparCC/lib/distances.py#L19

def JS(x,y): #Jensen-shannon divergence
    import warnings
    warnings.filterwarnings("ignore", category = RuntimeWarning)
    d1 = x*np.log2(2*x/(x+y))
    d2 = y*np.log2(2*y/(x+y))
    d1[np.isnan(d1)] = 0
    d2[np.isnan(d2)] = 0
    d = 0.5*sum(d1+d2)
    return d


JS(t11, p11), JS(t_large, p_large), JS(t_nan, p_nan), JS(t_neg, p_neg)

(7.275875413762115, 82030912.4664884, 6.109483922657932, -1486.8471425886949)

### Median Squared Error

In [103]:
from sktime.performance_metrics.forecasting import MedianSquaredError

mdse = MedianSquaredError()
mdse(t11, p11), mdse(t_large, p_large), mdse(t_neg, p_neg), # mdse(t_nan, p_nan)

(0.06731204476856545, 11033435538276.316, 3308.5)